# 示例：使用OSPFv3的Starlink卫星网络

## 概述
本示例构建一个卫星网络，拓扑为Starlink Shell5拓扑（36*20Grid拓扑）。每个卫星搭在一个星载路由器，运行OSPFv3协议。随后在卫星和地面站中宣告路由，观察路由传播情况，并使用ping观察星间、星地传输路径的时延。本示例可以帮助用户理解卫星网络的动态性对星间/星地传输路径时延的影响。

本示例中，运行两个示例实验：
1. 前端界面注入异常实验：通过前端界面使一条链路发生故障，观察网络路由收敛情况；
2. 脚本注入异常实验：通过脚本使一条链路发生故障，观察网络路由收敛情况；

## 示例网络配置
本示例使用Starlink Shell5子星座配置
### 星座及信关站配置
```json
// Starlink Shell 5
{
    "name": "my_constellation",
    "type": "walker_delta",
    "orbit_altitude": 570, //轨道高度（km）
    "orbit_inclination": 70, //轨道倾角（°）
    "orbit_num": 36, //轨道数量
    "sat_num_per_orbit": 20, //每轨道内卫星数量
    "phase_shift": 1,
    "sat_isl_link_num": 4,
    "sat_gsl_link_num": 1,
    "sat_access_link_num": 0,
}
```
```json
[
    // San Fransisco
    {
      "name": "gs_0",
      "latitude": 37.77,
      "longitude": -122.42,
      "elevation": 121.1,
      "gs_antenna_num": 1, //天线数量
      "gs_antenna_angle": 25, //天线最小仰角
    },
    // Washington
    {
      "name": "gs_1",
      "latitude": 38.91,
      "longitude": -77.01,
      "elevation": 126.1,
      "gs_antenna_num": 1, //天线数量
      "gs_antenna_angle": 25, //天线最小仰角
    },
]
```



### 卫星配置
+ 接口设置：共设置5个网络接口eth0-eth4。其中eth0-eth3是星间链路接口，对端目标是邻居卫星（具体参考[此文档](../../satnet/satnet_phy_engine.md#接口配置)）；eth4是星地链路接口，对端目标是信关站。
+ ip设置：设置lo地址为`fd01::x:y:1`，其中x为该卫星的轨道编号，y为该卫星在轨内的编号。
+ ospf设置：开启eth0-eth4的ospfv3功能，所有接口的area均设为backbone area(0.0.0.0)，即把所有星间、星地链路囊括在backbone area中。将ospf路由器id设置为`192.x.y.1`其中x为该卫星的轨道编号，y为该卫星在轨内的编号。通过`redistribute connected`将本卫星已经连接的路由（此示例中为通向本机lo地址的路由）通过ospf宣告给邻居。

### 信关站配置
+ 接口设置：设置1个星地链路接口eth0，即只有一根天线。
+ ip设置：设置lo地址为`fd02::x:1/128`，其中x为该信关站的编号。
+ ospf设置：开启eth0的ospfv3功能，area均设为backbone area(0.0.0.0)。将ospf路由器id设置为`172.16.x.1`其中x为该信关站的编号。通过`redistribute connected`将本信关站已经连接的路由（此示例中为通向本机lo地址的路由）通过ospf宣告给邻居。

## 操作步骤

In [4]:
API_KEY = 'b272c6a86e'
BASE_URL = f'https://{API_KEY}.backend.sernes.cn'
# BASE_URL = "http://10.10.30.158:9000"
# 网络配置
NETWORK_NAME = 'satnet_with_dhcp'

from tools import APIClient, WebSocketClient
import asyncio
import time

client = APIClient(BASE_URL)


### 步骤一：创建Starlink网络
- 在可视化界面中，点击“创建网络”。

- 在“网络信息”中，将实验名称改为starlink_test。

    <img src=images/net_info.png width=60% />

- 在“星座配置”中，按照下列参数修改网络配置（其他配置保持默认不变）：
    - 轨道高度：570
    - 轨道倾角：70
    - 轨道数量：36
    - 每卫星轨道数量：20

    <img src=images/const_info.png width=60% />

- 在“信关站配置”中，按照下列参数修改网络配置（其他配置保持默认不变）：
    - gs_0：纬度改为37.77，经度改为-122.42，天线角度改为25
    - gs_1：纬度改为38.91，经度改为-77.01，天线角度改为25

    <img src=images/gs_info.png width=60% />

- 在配置最后一步“上次配置zip包”页面中，上传示例配置文件[satnet_ospf_starlink_s5_redistribute.zip](satnet_ospf_starlink_s5_redistribute.zip)。

    <img src=images/zip.png width=60% />



In [2]:
# 创建网络的请求参数
network_params = {
  "network_name": "satnet_ospfv3",
  "network_description": "Satellite Network with OSPFv3",
  "constellation": {
    "name": "my_constellation",
    "type": "walker_delta",
    "orbit_altitude": 570,
    "orbit_inclination": 70,
    "orbit_num": 36,
    "sat_num_per_orbit": 20,
    "phase_shift": 1,
    "sat_isl_link_num": 4,
    "sat_gsl_link_num": 1,
    "sat_access_link_num": 0
  },
  "gs_set": [
    {
      "name": "gs_0",
      "latitude": 37.77,
      "longitude": -122.42,
      "elevation": 121.1,
      "gs_antenna_num": 1,
      "gs_antenna_angle": 25,
    },
    {
      "name": "gs_1",
      "latitude": 38.91,
      "longitude": -77.01,
      "elevation": 126.1,
      "gs_antenna_num": 1,
      "gs_antenna_angle": 25,
    }
  ],
}
# 发送创建网络的POST请求
print(network_params)
response = client.post('/api/network/create/', json_data=network_params)
if response:
    print("网络创建成功")
    network_id = response['network_id']
else:
    print("网络创建失败")

{'network_name': 'satnet_ospfv3', 'network_description': 'Satellite Network with OSPFv3', 'constellation': {'name': 'my_constellation', 'type': 'walker_delta', 'orbit_altitude': 570, 'orbit_inclination': 70, 'orbit_num': 36, 'sat_num_per_orbit': 20, 'phase_shift': 1, 'sat_isl_link_num': 4, 'sat_gsl_link_num': 1, 'sat_access_link_num': 0}, 'gs_set': [{'name': 'gs_0', 'latitude': 37.77, 'longitude': -122.42, 'elevation': 121.1, 'gs_antenna_num': 1, 'gs_antenna_angle': 25}, {'name': 'gs_1', 'latitude': 38.91, 'longitude': -77.01, 'elevation': 126.1, 'gs_antenna_num': 1, 'gs_antenna_angle': 25}]}
网络创建成功


In [3]:
client.post(f'/api/network/{network_id}/file/upload', files={'file': ('config.zip', open('satnet_ospf_starlink_s5_redistribute.zip', 'rb'))})

{'status': 'success',
 'path': '/mnt/reals-driver/nfs/netconf/network2/config_network2_20250508_145558.zip'}

### 步骤二：应用网络
在后台管理界面中，应用该网络，进入可视化界面。



In [7]:
# 发送启动网络的POST请求
client.get(f'/api/network/{network_id}/run/')
# 检查网络是否启动成功
while True:
    response = client.get(f'/api/network/list', {
        'id': network_id
    })
    network_status = response['data'][0]['network_status']
    if network_status == 1:
        print('网络启动成功')
        break
    time.sleep(1)

网络启动成功


### 步骤三：创建场景
在可视化界面中，点击“创建场景”，场景参数配置如下：

- 场景时间：按需设置（推荐300s以上）
- 异轨星间链路异常发生概率：0
- 日凌判定链路太阳光夹角：0

<img src=images/sc_time.png width=28% />
<img src=images/sc_prob.png width=28% />
<img src=images/sc_angle.png width=28% />

其他参数保持默认即可。创建场景后，等待该场景计算完成，点击播放按钮，运行该场景。

### 步骤四：验证星间路由连通性
场景启动后，测试两个卫星间的通信延迟。由于在[卫星配置](#卫星配置)和[信关站配置](#信关站配置)中已经将各节点的lo路由宣告出去，因此各卫星与信关站可以ping通各自的lo路由（信关站需要在有星地链路时才能ping通）。

点击任一卫星，进入其终端（以Sat123为例），输入如下命令查看路由：
```
bash-5.1# vtysh

Hello, this is FRRouting (version 8.4_git).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

Sat123# show ipv6 route
(Most routes are omitted...)
O>* fd01::35:17:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2, weight 1, 00:00:01
O   fd01::35:18:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2 inactive, weight 1, 00:00:00
O>* fd01::35:18:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2, weight 1, 00:00:01
O   fd01::35:19:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2 inactive, weight 1, 00:00:00
O>* fd01::35:19:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2, weight 1, 00:00:01
O * fd02::1/128 [110/20] via fe80::4033:40ff:fe2d:b592, eth3 inactive, weight 1, 00:00:00
O>q fd02::1/128 [110/20] via fe80::4033:40ff:fe2d:b592, eth3, weight 1, 00:00:01
O * fd02::1:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2 inactive, weight 1, 00:00:00
O>q fd02::1:1/128 [110/20] via fe80::1c38:36ff:fe54:9ea3, eth2, weight 1, 00:00:01
  q                        via fe80::b88a:34ff:fe22:c5e5, eth1, weight 1, 00:00:01
C * fe80::/64 is directly connected, eth3, 00:01:33
C * fe80::/64 is directly connected, eth1, 00:01:33
C * fe80::/64 is directly connected, eth0, 00:01:33
C * fe80::/64 is directly connected, eth4, 00:01:33
C>* fe80::/64 is directly connected, eth2, 00:01:33
```

发现其具有通向其他卫星与信关站lo地址的路由。

在Sat123上，向Sat321发送ping请求：
```
Sat123# ping fd01::16:1:1
PING fd01::16:1:1 (fd01::16:1:1): 56 data bytes
64 bytes from fd01::16:1:1: seq=0 ttl=53 time=82.606 ms
64 bytes from fd01::16:1:1: seq=1 ttl=53 time=82.065 ms
64 bytes from fd01::16:1:1: seq=2 ttl=53 time=82.716 ms
64 bytes from fd01::16:1:1: seq=3 ttl=53 time=82.127 ms
64 bytes from fd01::16:1:1: seq=4 ttl=53 time=82.614 ms
64 bytes from fd01::16:1:1: seq=5 ttl=53 time=82.363 ms
```
发现可以ping通。

### 步骤五：前端界面注入异常实验
在前端页面，点击Sat123周围的三条链路（上、右、下），点击“在线”按钮，将它们变为“离线”状态，模拟链路异常。此时卫星网络需要通过ospf重新计算路由，因此通信会中断一段时间。

<img src=images/fe_link_down.png width=60% />

断开链路时，观察Sat123向Sat321的ping情况，发现在seq=19后约30s的时间间隔内通信中断，证明ospf重新收敛过程中，Sat123至Sat321的路由不可用。随后在seq=51时通信重新建立，证明ospf重新收敛。然而由于收敛后的路径比源路径更长（需要通过左侧链路绕路），ping的时延相较于断链前有所增加。

```
64 bytes from fd01::16:1:1: seq=11 ttl=53 time=77.245 ms
64 bytes from fd01::16:1:1: seq=12 ttl=53 time=76.451 ms
64 bytes from fd01::16:1:1: seq=13 ttl=53 time=76.772 ms
64 bytes from fd01::16:1:1: seq=14 ttl=53 time=76.206 ms
64 bytes from fd01::16:1:1: seq=15 ttl=53 time=76.644 ms
64 bytes from fd01::16:1:1: seq=16 ttl=53 time=76.565 ms
64 bytes from fd01::16:1:1: seq=17 ttl=53 time=76.508 ms
64 bytes from fd01::16:1:1: seq=18 ttl=53 time=76.489 ms
64 bytes from fd01::16:1:1: seq=19 ttl=53 time=76.712 ms
64 bytes from fd01::16:1:1: seq=51 ttl=51 time=94.413 ms #异常！
64 bytes from fd01::16:1:1: seq=52 ttl=51 time=90.199 ms
64 bytes from fd01::16:1:1: seq=53 ttl=51 time=92.347 ms
64 bytes from fd01::16:1:1: seq=54 ttl=51 time=91.117 ms
64 bytes from fd01::16:1:1: seq=55 ttl=51 time=99.644 ms
```

此外，在路由变动监控界面中，可以看到实验中的路由变动情况。下图中，第一段曲线上升处的路由变动是网络启动过程中的ospf收敛造成的，第二段路由变动是断开上述三条链路造成的，证明手动注入的链路异常造成了全网ospf路由变动。

<img src=images/fe_down_panel.png width=60% />

### 步骤六：脚本注入异常实验
停止场景，重新运行该场景。在Sat123中执行`ping fd01::16:1:1`向Sat321发送ping请求。

在场景运行过程中，通过脚本[isl_batch_operation.py](isl_batch_operation.py)向卫星网络注入链路异常。在任意环境中（如用户自己的PC）执行：

In [7]:
API_KEY

'b272c6a86e'

In [11]:
!python3 isl_batch_operation.py down -r 0.2 --api-key $API_KEY

Link set operation succeeded.


In [12]:
!python3 isl_batch_operation.py up --api-key $API_KEY

Link set operation succeeded.


上述脚本执行后，网络中30%的链路被设置为down（不可用）。可以在前端看见如下画面，橙色链路为被设置为down的链路
<img src=images/batch_down.png width=40% />

注入异常后，卫星网络需要通过ospf重新计算路由，因此通信会中断一段时间。观察Sat123向Sat321的ping情况，发现在seq=50后约42s的时间间隔内通信中断，证明ospf重新收敛过程中，Sat123至Sat321的路由不可用。随后在seq=92时通信重新建立，证明ospf重新收敛。然而由于收敛后的路径比源路径更长（大量链路不可用导致需要新路径需要绕路），ping的时延相较于异常注入前有所增加。

```
64 bytes from fd01::16:1:1: seq=46 ttl=53 time=95.167 ms
64 bytes from fd01::16:1:1: seq=47 ttl=53 time=94.949 ms
64 bytes from fd01::16:1:1: seq=48 ttl=53 time=95.522 ms
64 bytes from fd01::16:1:1: seq=49 ttl=53 time=94.697 ms
64 bytes from fd01::16:1:1: seq=50 ttl=53 time=94.079 ms
64 bytes from fd01::16:1:1: seq=92 ttl=49 time=179.849 ms #异常！
64 bytes from fd01::16:1:1: seq=94 ttl=49 time=131.151 ms
64 bytes from fd01::16:1:1: seq=95 ttl=49 time=156.219 ms
64 bytes from fd01::16:1:1: seq=96 ttl=51 time=188.035 ms
64 bytes from fd01::16:1:1: seq=97 ttl=49 time=151.281 ms
64 bytes from fd01::16:1:1: seq=98 ttl=49 time=143.217 ms
```

此外，在路由变动监控界面中，可以看到实验中的路由变动情况。下图中，第一段曲线上升处的路由变动是网络启动过程中的ospf收敛造成的，第二段路由变动则是脚本注入链路异常造成的。

<img src=images/batch_down_panel.png width=60% />

## 配置文件解读

### config.yaml配置文件
`nodes`字段包含卫星和信关站两种节点，每个节点为一个运行OSPFv3的路由器，使用[FRRouting](https://frrouting.org/)软件路由器实现。
每个卫星节点包含6个`fixed_vNICs`，其中eth0-eth3是星间链路接口，对端目标是邻居卫星（具体参考[此文档](../../satnet/satnet_phy_engine.md#接口配置)）；eth4是星地链路接口，对端目标是信关站。
每个信关站节点包含1个`fixed_vNICs`，即eth0。

在`links`字段中，使用`walker_delta`引擎将所有卫星连接成30*30的Grid拓扑。星地链路则由[最近最长维持切换策略](../../satnet/satnet_phy_engine.md#星地链路建立策略)动态建立/断开。

```yaml
templates:
  sat_tmpl:
    node_type: sat
    image: library/frrouting/frr:v8.4.0
    start_commands:
    - chown -R frr:frr /var/log/frr
    - /usr/lib/frr/frrinit.sh start
    stop_commands:
    - /usr/lib/frr/frrinit.sh stop
    volumes:
    - frr_conf:/etc/frr
    - frr_log:/var/log/frr
    fixed_vNICs:
      eth0: {}
      eth1: {}
      eth2: {}
      eth3: {}
      eth4: {}
  gs_tmpl:
    node_type: gs
    image: library/frrouting/frr:v8.4.0
    start_commands:
    - chown -R frr:frr /var/log/frr
    - /usr/lib/frr/frrinit.sh start
    stop_commands:
    - /usr/lib/frr/frrinit.sh stop
    volumes:
    - frr_conf:/etc/frr
    - frr_log:/var/log/frr
fixed_nodes:
  Sat0:
    template: sat_tmpl
    sat_id: 0
  Sat1:
    template: sat_tmpl
    sat_id: 1
  (Other satellites are omitted)
  gs_0:
    template: gs_tmpl
    gs_id: 0
    fixed_vNICs:
      eth0: {}
  gs_1:
    template: gs_tmpl
    gs_id: 1
    fixed_vNICs:
      eth0: {}
  (Other ground stations are omitted)
  (No end users in this example)
links:
  engines:
  - walker_delta
  veth_pairs: []
```

### 路由节点配置文件
以Sat0为例，
zebra.conf如下：
```
interface lo
  ipv6 address fd01::0:0:1/128
ip forwarding
ipv6 forwarding
log timestamp precision 6
log file /var/log/frr debug
```

ospf6d.conf如下：
```
!
interface eth0
    ipv6 ospf6 area 0.0.0.0
exit
!
interface eth1
    ipv6 ospf6 area 0.0.0.0
exit
!
interface eth2
    ipv6 ospf6 area 0.0.0.0
exit
!
interface eth3
    ipv6 ospf6 area 0.0.0.0
exit
!
interface eth4
    ipv6 ospf6 area 0.0.0.0
exit
!
router ospf6
    ospf6 router-id 192.0.0.1
    redistribute connected
exit
!
log timestamp precision 6
log file /var/log/frr/ospf6d.log debug
```

上述配置文件中，根据[卫星配置](#卫星配置)章节配置该卫星：
+ zebra.conf中，`ipv6 address fd01::0:0:1/128`为该卫星的lo接口配置一个ipv6地址`fd01::0:0:1/128`；
+ ospf6d.conf中，通过`ipv6 ospf6 area 0.0.0.0`将eth0-eth4——即星间、星地链路网口设置为开启ospfv3的网口。所有接口的area均设为backbone area(0.0.0.0)。`ospf6 router-id 192.0.0.1`将本卫星的ospf路由器id设置为`192.0.0.1`。`redistribute connected`将本卫星已经连接的路由（此示例中为通向本机lo地址的路由）通过ospf宣告给邻居。